In [40]:
import pandas as pd
import json
from sqlalchemy import create_engine 
from pandas.io.json import json_normalize

In [41]:
yt_file = "./youtube_files/USvideos.csv"
usvideos_df = pd.read_csv(yt_file)
usvideos_df = pd.DataFrame(usvideos_df)
usvideos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [42]:
type(usvideos_df)

pandas.core.frame.DataFrame

In [43]:
usvideos_df.dtypes

video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

In [44]:
usvideos_df.rename(columns = {"channel_title": "channel_name"}, inplace = True)

In [45]:
usvideos_df['trending_date'] = pd.to_datetime(usvideos_df['trending_date'], format='%y.%d.%m')
usvideos_df['trending_date'].head()

0   2017-11-14
1   2017-11-14
2   2017-11-14
3   2017-11-14
4   2017-11-14
Name: trending_date, dtype: datetime64[ns]

In [46]:
usvideos_df['publish_time'] = pd.to_datetime(usvideos_df['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
usvideos_df['publish_time'].head()

0   2017-11-13 17:13:01
1   2017-11-13 07:30:00
2   2017-11-12 19:05:24
3   2017-11-13 11:00:04
4   2017-11-12 18:01:41
Name: publish_time, dtype: datetime64[ns]

In [47]:
usvideos_df.insert(4, 'publish_date',usvideos_df['publish_time'].dt.date)
usvideos_df['publish_time'] = usvideos_df['publish_time'].dt.time
usvideos_df[['publish_date', 'publish_time']].head()

,publish_date,publish_time
0,2017-11-13,17:13:01
1,2017-11-13,07:30:00
2,2017-11-12,19:05:24
3,2017-11-13,11:00:04
4,2017-11-12,18:01:41


In [48]:
usa_trending_videos_df = usvideos_df.drop(['comments_disabled', 'ratings_disabled', 
                                 'video_error_or_removed', 'tags' , 'description', 'thumbnail_link' ], axis=1)
usa_trending_videos_df.head()

,video_id,trending_date,title,channel_name,publish_date,category_id,publish_time,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,748374,57527,2966,15954
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,2418783,97185,6146,12703
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,3191434,146033,5339,8181
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,343168,10172,666,2146
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,2095731,132235,1989,17518


In [49]:
type(usa_trending_videos_df)

pandas.core.frame.DataFrame

In [50]:
columns = ['views', 'likes', 'dislikes', 'comment_count']
for column in columns:
    usa_trending_videos_df[column] = usa_trending_videos_df[column].astype(int)

columns = ['category_id']
for column in columns:
    usa_trending_videos_df[column] = usa_trending_videos_df[column].astype(str)

In [51]:
usa_trending_videos_df.dtypes

video_id                 object
trending_date    datetime64[ns]
title                    object
channel_name             object
publish_date             object
category_id              object
publish_time             object
views                     int32
likes                     int32
dislikes                  int32
comment_count             int32
dtype: object

In [25]:
usa_trending_videos_df.to_csv(r'C:\Users\Jose\Documents\youtube_etl_project\us_trending_videos_interactions.csv')

In [52]:
usa_trending_videos_df.head()

,video_id,trending_date,title,channel_name,publish_date,category_id,publish_time,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,748374,57527,2966,15954
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,2418783,97185,6146,12703
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,3191434,146033,5339,8181
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,343168,10172,666,2146
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,2095731,132235,1989,17518


In [53]:
json_file = "./youtube_files/US_category_id.json"
json_df = pd.read_json(json_file)

json_df.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [54]:
# the dataframe from the json is not helpful so I am prettyprinting it to make it more readable. 

#Mark's code 
#json_file = "./youtube_files/US_category_id.json"
#json_parsed = None;
#with open(json_file) as json_data:
    #json_parsed = json.load(json_data)

with open('./youtube_files/US_category_id.json', 'r') as json_data:
    data = json.load(json_data)
    print(json.dumps(data, indent=4, sort_keys=True))

{
    "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
    "items": [
        {
            "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
            "id": "1",
            "kind": "youtube#videoCategory",
            "snippet": {
                "assignable": true,
                "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
                "title": "Film & Animation"
            }
        },
        {
            "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA\"",
            "id": "2",
            "kind": "youtube#videoCategory",
            "snippet": {
                "assignable": true,
                "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
                "title": "Autos & Vehicles"
            }
        },
        {
            "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg\"",
            "id": "10",
            "kind": "youtube#videoCategory",
            "snippet": {
                "assignab

In [55]:
#seeing the json data I can extract the info I need using the following:

category_name = {}


with open('./youtube_files/US_category_id.json', 'r') as json_data:
    data = json.load(json_data)
    for category in data['items']:
        category_name[category['id']] = category['snippet']['title']

category_name 

# in the csv there is a category_id. In the json file you can find what category name the category_id refers to.
# I used this function to extract it. 

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '29': 'Nonprofits & Activism',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [56]:
#adding it to my dataframe
usa_trending_videos_df.insert(4, 'category', usa_trending_videos_df['category_id'].map(category_name))
usa_trending_videos_df[['category_id', 'category']].head()

,category_id,category
0,22,People & Blogs
1,24,Entertainment
2,23,Comedy
3,24,Entertainment
4,24,Entertainment


In [57]:
usa_trending_videos_df.head()

,video_id,trending_date,title,channel_name,category,publish_date,category_id,publish_time,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,2017-11-13,22,17:13:01,748374,57527,2966,15954
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13,24,07:30:00,2418783,97185,6146,12703
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12,23,19:05:24,3191434,146033,5339,8181
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,2017-11-13,24,11:00:04,343168,10172,666,2146
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12,24,18:01:41,2095731,132235,1989,17518


In [60]:
usa_trending_videos_df = usa_trending_videos_df.dropna()
usa_trending_videos_df.head()

,video_id,trending_date,title,channel_name,category,publish_date,category_id,publish_time,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,2017-11-13,22,17:13:01,748374,57527,2966,15954
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13,24,07:30:00,2418783,97185,6146,12703
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12,23,19:05:24,3191434,146033,5339,8181
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,2017-11-13,24,11:00:04,343168,10172,666,2146
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12,24,18:01:41,2095731,132235,1989,17518


In [58]:
type(usa_trending_videos_df)

pandas.core.frame.DataFrame

In [61]:
usa_trending_videos_df.dtypes

video_id                 object
trending_date    datetime64[ns]
title                    object
channel_name             object
category                 object
publish_date             object
category_id              object
publish_time             object
views                     int32
likes                     int32
dislikes                  int32
comment_count             int32
dtype: object

In [62]:
usa_trending_videos_df.to_csv(r'C:\Users\Jose\Documents\youtube_etl_project\usa_trending_videos_interactions.csv')

#usa_trending_videos_df.to_csv(r'C:\Users\Jose\Documents\youtube_etl_project\us_trending_videos_interactions.csv')